# Amazon Bedrock Guardrails configuration

> *This notebook should work well with the **`Python 3 (ipykernel)`** kernel in SageMaker Studio on ml.t3.medium instance*

In this lab, we will perform the following tasks to build and test a guardrail configuration.

1. Define the name and the description
2. Define the tone filter levels for different content types
3. Define a denied topic to create an operational boundary 
4. Define a list of prohibited words that we don't want to allow in the user prompt and the LLM's response
5. Define the personally identifiable information (PII) detection and masking
6. Define a contextual grounding check policy
7. Define the default responses if the guardrail blocks a request or a response
8. Create a guardrail configuration using the Bedrock API
9. Verify the guardrail configuration created in Bedrock

We have a lot to cover. So, let's jump in!

### Validate the environment

First, we will import required libraries and validate the environment sanity.

In [ ]:
import sys
import os
import json
import boto3
import time

from pathlib import Path
from rich import print as rprint
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

Now, we will create and test Bedrock client connection.

In [ ]:
# Create the bedrock client for model access
bedrock_client = boto3.client("bedrock")

# Validate bedrock client connection
if bedrock_client is not None:
    rprint("Successfully connected to Bedrock")

# Test Bedrock client connection by listing available guardrails in the account for current region
try:
    guardrails = bedrock_client.list_guardrails()
    rprint("Successfully retrieved available guardrails.")
    rprint(guardrails)

except Exception as e:
    rprint(f"Error creating guardrail: {str(e)}")
    raise

Depending on the state of your account, you might not see listed guardrails in the response. However, a successful response validates the connection for the Bedrock client. 

### Define the name and the description

Now, we will define various attributes and configuration items that we will use for our Bedrock Guardrails configuration.

In [ ]:
epoch_int = int(time.time())

# Define metadata
guardrail_name = 'BasicGuardrail-' + str(epoch_int)
guardrail_description = 'Basic guardrail configuration to be ' + \
                        'used with GenAI boot camp lab'
rprint(f"Guardrail Name: {guardrail_name}")

### Define the tone filter levels

Now, let's define a high-level tolerance boundary for hateful and insulting tones.

In [ ]:
# Define tone filter strengths for hateful and insulting content

guardrail_filters = {
                'filtersConfig': [
                    {
                        'inputStrength': 'HIGH',
                        'outputStrength': 'HIGH',
                        'type': 'HATE',
                        'inputModalities': ['TEXT', 'IMAGE'],
                        'outputModalities': ['TEXT', 'IMAGE']
                    },
                    {
                        'inputStrength': 'MEDIUM',
                        'outputStrength': 'MEDIUM',
                        'type': 'INSULTS',
                        'inputModalities': ['TEXT', 'IMAGE'],
                        'outputModalities': ['TEXT', 'IMAGE']
                    },
                    {
                        'type': 'VIOLENCE',
                        'inputStrength': 'HIGH',
                        'outputStrength': 'HIGH',
                        'inputModalities': ['TEXT', 'IMAGE'],
                        'outputModalities': ['TEXT', 'IMAGE']
                    },
                    {
                        'type': 'PROMPT_ATTACK',
                        'inputStrength': 'HIGH',
                        'outputStrength': 'NONE',
                        'inputModalities': ['TEXT'],
                        'outputModalities': ['TEXT']
                    }
                ]}

In the previous configuration, we have just configured the filters for hateful and insulting content. However, Bedrock Guardrails also supports other tones including misconduct, sexuality, and violence. 

### Define a denied topic
In the following configuration, we will define certain topics that we don't want to entertain. 

In [ ]:
# Define a denied topic to decline any asks for a financial advice with a couple of examples

guardrail_deny_topics = {
                'topicsConfig': [
                    {
                        'name': 'Financial Advice',
                        'definition': 'Providing financial or investment advice',
                        'examples': [
                            'What stocks should I invest in?',
                            'How should I invest my money?'
                        ],
                        'type': 'DENY'
                    }
                ]
            }

Though the previous configuration includes only one deny topic, you may add more topics in the list. As of this writing, you can only configure the `DENY` topics.

### Define a list of prohibited words
Now, let's configure a word exclusion policy to list certain words that we want to filter.

In [ ]:
# Define a word exclusion policy

guardrail_deny_words = {
                'wordsConfig': [
                    {
                        'text': 'idiot'
                    },
                    {
                        'text': 'gun'
                    }
                ],
                'managedWordListsConfig': [
                    {
                        'type': 'PROFANITY'
                    }
                ]
            }

In the previous example, we listed two specific words that we want to deny - `idiot` and `gun`. Other than such specific words that you want to deny, Bedrock maintains a list of words in a specific area - profanity. As of this writing, this is the only supported list of words offered by Bedrock. 

### Define a PII handling policy
Now, let's configure information redaction policy to mask certain sensitive financial PII details like a credit card number, a credit card CVV, and a credit card expiry date.

In [ ]:
# Define sensitive information policy

guardrail_redact_data = {
                'piiEntitiesConfig': [
                    {
                        'type': 'CREDIT_DEBIT_CARD_NUMBER',
                        'action': 'ANONYMIZE'
                    },
                    {
                        'type': 'CREDIT_DEBIT_CARD_CVV',
                        'action': 'BLOCK'
                    },
                    {
                        'type': 'CREDIT_DEBIT_CARD_EXPIRY',
                        'action': 'BLOCK'
                    }
                ]
            }

In the previous configuration, we only configured three PII data elements. However, Bedrock Guardrails supports several such data types, including but not limited to names, addresses, zipcode, phone number, email address, etc. For a full list of configuration please check [this website](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_GuardrailPiiEntityConfig.html). Additionally, you can also define your own data elements for this purpose using their regex patterns as described in [this website](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_GuardrailRegexConfig.html).

### Define a contextual grounding check policy

Now, let's define a contextual grounding check policy that will check model's responses to see if they are faithful to give context data and relevant to the request sent.

In [ ]:
# Define contextual grounding check policy

contextual_grounding_policy = {
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.7
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.7
            }
        ]
    }

### Define the default responses for the blocked content
Now, let's define how to handle all these policy exceptions when we encounter any denied content in the request and the response generated by an LLM.

In [ ]:
# Define what should the users get when their 
# request is blocked by our guardrail configuration.
guardrail_blocked_input_message = (
                    'Sorry, but I cannot help you with this topic '
                    'as it violates my responsible behavior policy.'
)

# Define what should the users get when the LLM generates
# some content that violates our guardrail configuration.
guardrail_blocked_output_message = (
                    'Sorry, but I cannot generate a response to your '
                    'ask that does not violate my responsible behavior policy.'
)

### Create a guardrail configuration in Bedrock
Now, as we have all required configurations to create our solid guardrail policy, let's create one using the Bedrock client we created in the beginning of the lab.

In [ ]:
# Created a Bedrock Guardrail configuration
guardrail_id = None
guardrail_arn = None
guardrail_version = None
try:
    guardrail_response = bedrock_client.create_guardrail(
        name=guardrail_name,
        description=guardrail_description,
        contentPolicyConfig=guardrail_filters,
        topicPolicyConfig=guardrail_deny_topics,
        wordPolicyConfig=guardrail_deny_words,
        sensitiveInformationPolicyConfig=guardrail_redact_data,
        blockedInputMessaging=guardrail_blocked_input_message,
        blockedOutputsMessaging=guardrail_blocked_output_message,
        contextualGroundingPolicyConfig=contextual_grounding_policy
    )
    guardrail_id = guardrail_response['guardrailId']
    guardrail_arn = guardrail_response['guardrailArn']
    # Create a version of the guardrail
    version_response = bedrock_client.create_guardrail_version(
        guardrailIdentifier=guardrail_id,
        description='Initial version'
    )
    guardrail_version = version_response['version']
except Exception as e:
    rprint(f"Error creating guardrail: {str(e)}")
    raise

rprint(f"Guardrail created successfully with ID \"{guardrail_id}\" and version \"{guardrail_version}\".")

Let's store the guardrail identifiers in a file, so that we can use them in the next labs.

In [ ]:
# save guardrail details in first notebook
Path("guardrail_config.json").write_text(
    json.dumps(
        {
            "guardrail_id": guardrail_id,
            "guardrail_arn": guardrail_arn,
            "guardrail_version": guardrail_version
        }
    )
)


```{note}
Please note the guardrail ID from the last line of the output. We will use this ID to pull this configuration for validations in the upcoming labs.
```

### Verify the guardrail configuration created in Bedrock
Now, as we created the guardrail successfully, let's verify the configuration of the same.

In [ ]:
# Get a Bedrock Guardrail configuration with its ID and version
guardrail = None
try:
    # Get the guardrail configuration
    guardrail = bedrock_client.get_guardrail(
        guardrailIdentifier=guardrail_id,
        guardrailVersion=guardrail_version
    )

    # Describe details of the guardrail configuration
    if guardrail is not None:
        # Print guardrail details
        rprint("\nGuardrail Configuration:")
        rprint(f"Name: {guardrail['name']}")
        rprint(f"Description: {guardrail['description']}")

        # Print policy configurations if they exist
        rprint("===================================================")
        if 'contentPolicy' in guardrail:
            rprint("\nContent Policy Configuration:")
            rprint(json.dumps(guardrail['contentPolicy'], indent=2))

        rprint("===================================================")
        if 'topicPolicy' in guardrail:
            rprint("\nTopic Policy Configuration:")
            rprint(json.dumps(guardrail['topicPolicy'], indent=2))

        rprint("===================================================")
        if 'wordPolicy' in guardrail:
            rprint("\nWord Policy Configuration:")
            rprint(json.dumps(guardrail['wordPolicy'], indent=2))

        rprint("===================================================")
        if 'sensitiveInformationPolicy' in guardrail:
            rprint("\nSensitive Information Policy Configuration:")
            rprint(json.dumps(guardrail['sensitiveInformationPolicy'],
                              indent=2))
        rprint("===================================================")
        if 'contextualGroundingPolicy' in guardrail:
            rprint("\nContextual Grounding Policy Configuration:")
            rprint(json.dumps(guardrail['contextualGroundingPolicy'],
                              indent=2))

except Exception as e:
    rprint(f"Error getting guardrail configuration: {str(e)}")
    raise

### Next Steps
In this lab, you learned how to create a Bedrock Guardrails configuration with different policies. In the next lab, we will test this guardrail configuration to validate a user prompt independent of a large language model.